In [1]:
import pandas as pd
import requests
import json
import xmltodict
import geopandas as gpd
import folium

from pandas import json_normalize

In [2]:
spot = pd.read_csv('data/tour_path.csv')
spot.head()

,spot,x,y
0,숭례문,126.975307,37.559984
1,덕수궁,126.975190,37.565537
2,이상의집,126.970876,37.578624
3,통인시장,126.969972,37.580694
4,박노수 미술관,126.966775,37.581279


In [3]:
REST_API_KEY = '80b269050cd58c9743d68720ddc84692'
tour_all = pd.DataFrame()

headers = {
    'Authorization': f"KakaoAK {REST_API_KEY}",
    'Content-Type': 'application/x-www-form-urlencoded',
}

for x, y, name in zip(spot['x'].tolist(), spot['y'].tolist(), spot['spot'].tolist()):
    params = {
        'y': f'{x}',
        'x': f'{y}',
        'radius': '500',
        'category_group_code': 'FD6',
    }

    data = 'query=맛집'

    response = requests.get('https://dapi.kakao.com/v2/local/search/keyword.json', headers=headers, params=params, data=data.encode('utf-8'))
    text = response.json()
    tour_list = json_normalize(text['documents'])
    tag = ['맛집' for _ in range(len(tour_list))]
    tour_list['tag'] = tag
    tour_all = pd.concat([tour_all, tour_list], axis=0)

    params = {
        'y': f'{x}',
        'x': f'{y}',
        'radius': '500',
        'category_group_code': 'CE7',
    }

    response = requests.get('https://dapi.kakao.com/v2/local/search/keyword.json', headers=headers, params=params, data=data.encode('utf-8'))
    text = response.json()
    tour_list = json_normalize(text['documents'])
    tag = ['카페' for _ in range(len(tour_list))]
    tour_list['tag'] = tag
    tour_all = pd.concat([tour_all, tour_list], axis=0)
    

In [4]:
tour_all.head()

,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,tag
0,서울 중구 무교동 97,FD6,음식점,음식점 > 양식,,444516464,02-6273-0100,디라이프스타일 키친 광화문점,http://place.map.kakao.com/444516464,서울 중구 세종대로 136,126.978257891095,37.5684224971952,맛집
1,서울 중구 다동 177-2,FD6,음식점,음식점 > 한식,,1346301870,02-777-4323,부민옥,http://place.map.kakao.com/1346301870,서울 중구 다동길 24-12,126.98031824515,37.5675038430283,맛집
2,서울 중구 을지로1가 192-11,FD6,음식점,"음식점 > 일식 > 돈까스,우동",,220828644,02-772-9994,이나니와요스케 서울시청점,http://place.map.kakao.com/220828644,서울 중구 을지로 6,126.97907374358543,37.5653376347315,맛집
3,서울 중구 다동 121-4,FD6,음식점,음식점 > 한식 > 냉면,,21232315,02-777-3131,남포면옥,http://place.map.kakao.com/21232315,서울 중구 을지로3길 24,126.981753597164,37.5671652991647,맛집
4,서울 중구 정동 1-48,FD6,음식점,음식점 > 한식 > 국밥,,2067124772,02-738-5688,광화문국밥 본점,http://place.map.kakao.com/2067124772,서울 중구 세종대로21길 53,126.975391668232,37.5689589279236,맛집


In [5]:
del tour_all['distance']
del tour_all['category_group_code']

tour_all = tour_all.drop_duplicates(['place_name'])

In [81]:
tour_all.to_csv('data/cafeteria.csv', index=False, encoding='cp949')

In [6]:
from folium.plugins import MarkerCluster

m = folium.Map(
    location=[37.571002, 126.976932],
    zoom_start=15
)

coords = tour_all[['x', 'y', 'place_name']]


marker_cluster = MarkerCluster().add_to(m)

for long, lat, name in zip(coords['x'], coords['y'], coords['place_name']):
    folium.Marker([lat, long], icon = folium.Icon(color="green"), tooltip=name).add_to(marker_cluster)
m